# Device mapping

Put device names in element alias field.

The master mapping of device name to element name is from the SLACPROD Oracle Database, downloaded as a CSV file:

https://oraweb.slac.stanford.edu/apex/slacprod/f?p=116:600


In [2]:
import pandas as pd
import numpy as np
import json
import os

# SLACPROD Oracle Table

In [3]:
#%env LCLS_LATTICE=/home/mpe/repos/lcls-lattice
os.path.expandvars('$FACET2_LATTICE')

'/home/mpe/repos/facet2-lattice/'

In [4]:
# Table extracted from SLACPROD Oracle Database
#%env LCLS_LATTICE=/home/mpe/code/lcls-lattice
MASTER = '$FACET2_LATTICE/bmad/conversion/from_oracle/lcls_elements.csv'

df = pd.read_csv(os.path.expandvars(MASTER))

# Remove empty
df = df[['Element', 'Control System Name']].dropna()

In [5]:
# These are unique
MADNAMES = list(df['Element'])
len(MADNAMES), len(set(MADNAMES))

(3699, 3699)

In [6]:
# These are not
DEVICENAMES = list(df['Control System Name'])
len(DEVICENAMES), len(set(DEVICENAMES))

(3699, 3286)

In [7]:
# These devices have multiple elements - a mistake?
series  = df.groupby('Control System Name')['Element'].apply(list)
for i, val in series.items():
    if len(val) > 1:
        # Skip klystrons - these are expected to be duplicated
        if not val[0].startswith('K'):
            print(i, val)

In [8]:
# dict for lookup
DEVICE = dict(zip(MADNAMES, DEVICENAMES))
json.dump(DEVICE, open('element_devices.json', 'w'))

# Models 

In [10]:
BDIR = os.path.expandvars('$FACET2_LATTICE/bmad/')

In [21]:
# All models
MODELS = [d for d in os.listdir(BDIR+'models/') if os.path.isdir(BDIR+'/models/'+d)]
INITFILE = {model:f'$FACET2_LATTICE/bmad/models/{model}/tao.init' for model in MODELS}
INITFILE

{'FACET2e': '$FACET2_LATTICE/bmad/models/FACET2e/tao.init',
 'f2_elec': '$FACET2_LATTICE/bmad/models/f2_elec/tao.init',
 'f2_posi': '$FACET2_LATTICE/bmad/models/f2_posi/tao.init',
 'f2e_inj': '$FACET2_LATTICE/bmad/models/f2e_inj/tao.init',
 'FACET2p_DR': '$FACET2_LATTICE/bmad/models/FACET2p_DR/tao.init'}

# PyTao

In [14]:
from pytao import Tao
import os

In [15]:
def ele_names(model):
    init = INITFILE[model]
    tao = Tao(f'-init {init} -noplot')
    names = tao.cmd('python lat_list 1@0>>*|model ele.name')
    return names

def remove_superslaves(names):
    return [x for x in names if '#' not in x]

In [16]:
def write_devicenames(unames, filename):
    my_names = remove_superslaves(unames)
    lines = ['! ---------',
             '! Device mapping derived from '+MASTER
             
            ]
    for name in my_names:
        if name in DEVICE:
            line = name+'[alias]='+ DEVICE[name]
            
        else:
            #continue
            line = '! No device listed for: '+name
        lines.append(line)    
    with open(filename, 'w') as f:
        for line in lines:
            f.write(line+'\n')
    print('Written:', filename)

# Add to FACET-II

In [28]:
if os.path.exists(FDIR):
    F2_FILE = f'{FDIR}/master/FACET2e_devicenames.bmad'
    models = ['f2_elec']
    names = []
    for m in models:
        print(m)
        names += ele_names(m)
    unames = sorted(list(set(names)))
    unames
    write_devicenames(unames, F2_FILE)

f2_elec
Written: /home/mpe/repos/facet2-lattice//bmad//master/FACET2e_devicenames.bmad


# elementdevices (old method)

In [20]:
#
#
#    ELEMENTDEVICES = os.path.expandvars('$LCLS_LATTICE/mad/elementdevices.dat')
#    os.path.exists(ELEMENTDEVICES)
#    
#    def parse_elementdevices(elementdevices_filename):
#        """
#        
#        Parameters
#        ----------
#        elementdevices_filename
#        
#        Returns
#        -------
#        device: dict of ele_name:devicename
#        not_found: list of ele_names with no device
#        
#        """
#        device = {}
#        not_found = []
#        with open(elementdevices_filename) as f:
#            for line in f:
#                x = line.split()
#                if len(x) != 2:
#                    continue
#    
#                ele, devicename = x
#                if devicename == '-':
#                    not_found.append(ele)
#                    continue
#                if ele in device:
#                    raise ValueError('ele already has a a device:', ele, device[ele])
#    
#                device[ele] = devicename   
#                
#        return device, not_found
#    DNAME, NOT_FOUND = parse_elementdevices(ELEMENTDEVICES)    
#    len(list(DNAME)), len(NOT_FOUND)

In [21]:
## Check for missing or mismatched items
#for ele, dev in DNAME.items():
#    if ele not in DEVICE:
#        #continue
#        print('Missing from Oracle table:', ele, dev)
#    else:
#        oracle_dev = DEVICE[ele] 
#        if oracle_dev != dev:
#        #    continue
#            print('Device mismatch for ele, oracle, elementdevices.dat:', ele, oracle_dev, dev)